Wir verwenden das Paket TextBlobDE, da er vortrainierte Modelle für deutsche Sentiment-Analyse enthält.

In [1]:
import pandas as pd
from textblob_de import TextBlobDE as TextBlob
from textblob.classifiers import NaiveBayesClassifier
import re
import nltk

Einlesen der gecleanten Cases als csv Datei

In [3]:
#df = pd.read_csv("./cleaned_cases.csv", sep="\t")
df = pd.read_csv("C:/Users/Sebastian/Desktop/Uni/NLP/NLPSTUDA/23-12-cleaned_cases.csv", sep="\t")

Wir verwenden einen NaiveBayesClassifier um das "Sentiment" der Cases zu bestimmen. Dabei bestimmen wir letztendlich den erwarteten Erfolg des Antrags aus den ersten Sätzen. Dafür sind wir den Text durchgegangen um einige typische negative und positive Formulierungen zu finden. Diese können wir für den NaiveBayesClassifier verwenden. 

In [4]:
negative = [
    "Der Antrag des Klägers wird abgelehnt",
    "Der Antrag wird auf Kosten der Klägerin abgelehnt",
    "Im Übrigen wird der Antrag abgelehnt",
    "Der Antrag wird abgelehnt",
    "Die Klage wird abgewiesen",
    "Der Kläger trägt die Kosten des Verfahrens",
    "Der Kläger trägt die Kosten des Zulassungsverfahrens",
    "Die Kläger tragen die Kosten des Zulassungsverfahrens",
    "Der Antrag der Kläger auf Zulassung der Berufung hat keinen Erfolg",
    "Die Beklagte trägt die Kosten",
    "schuldig",
    "verklagt",
    "Im Übrigen wird der Antrag abgelehnt",
    "Die Beschwerde wird zurückgewiesen",
    "Der Antragsteller trägt die Kosten des Beschwerdeverfahrens",
    "Berufung ist nicht zuzulassen",
    "Die Berufung der Klägerin gegen das Urteil der wird zurückgewiesen",
    "Die Kosten des Berufungsverfahrens trägt die Klägerin"
]

positive = [
    "Wird der Beschluss aufgehoben",
    "Die Kosten des Beschwerdeverfahrens werden der Staatskasse auferlegt",
    "Die Kosten fallen der Staatskasse zur Last",
    "Das Verfahren wird eingestellt",
    "stattgegeben",
    "eingestellt",
    "unschuldig",
    "Das Urteil wird aufgehoben",
    "Die Kosten trägt der Staat",
    "freigesprochen",
    "Der Antragsgegner trägt die Kosten",
]

train = []

for p in positive:
    train.append((p, 1))

for n in negative:
    train.append((n, -1))

Positive Sätze werden mit 1 bewerten, negative mit -1. Damit wird der Classifier trainiert

In [5]:
cl = NaiveBayesClassifier(train)

Zudem erfassen wir über eine RegEx Geldbeträge 

In [6]:
df["sentiment"] = 0
df["geldbetrag"] = None

In [7]:
regExGeld = r"(\d{0,3}\.?)\d{0,3},\d{2,3}\s?(Mrd\.\s)?\€"


Diese Funktion wird auf jede Zeile des DataFrames angewendet. Sie berechnet für die ersten drei Sätze das Sentiment mit dem NaiveBayesClassifier und bildet daraus den Mittelwert, der in die Zeile geschrieben wird. Diese Methode wurde durch Ausprobieren optimiert. Mit 3 Sätzen schien das Ergebnis am besten, da die Gerichtstexte ähnlich aufgebaut sind und der NaiveBayesClassifier wenig Trainingsdaten verwendet, die hauptsächlich auf diesem Bereich basieren.
Außerdem werden über die Regex alle im Text vorkommenden Geldbeträge gesucht und in die Zeile geschrieben. 

In [13]:
def calculateSentimentAndFindMoney(row):
    b= TextBlob(str(row.content))
    numberOfSentences = 3
    if len(b.sentences) >= numberOfSentences:
        first = b.sentences[1:numberOfSentences]
        sents = [s.raw for s in first]
    else:
        sents = [s.raw for s in b.sentences]
    allSentsSum = 0
    for s in sents:
        allSentsSum += cl.classify(s)
    
    row.sentiment= allSentsSum / len(sents)


    geldbetraege = [x.group() for x in re.finditer(pattern=regExGeld, string=str(row.content))]
    row.geldbetrag = geldbetraege
    return row

Anwenden der Funktion auf den Datensatz, zeilenweise

In [14]:
df = df.apply(calculateSentimentAndFindMoney, axis=1)

In [13]:
df.geldbetrag

0                                          [6.150,23 €]
1                                         [15.000,00 €]
2                                                    []
3                                                    []
4                                        [915.000,00 €]
                            ...                        
95                                                   []
96                                                   []
97                                                   []
98    [12.125,97 €, 1.180,50 €, 20.000,00 €, 16.572,...
99                                                   []
Name: geldbetrag, Length: 100, dtype: object

Abschließendes Speichern als csv

In [15]:
df.to_csv("C:/Users/Sebastian/Desktop/Uni/NLP/23-12_cleaned_cases_with_sentiments.csv")